# steps:
### 1 for the og df, creat a **winner, loser, winner conf, loser conf, in conf? _columns_**
### 2 <span style="color:red">melt</span> to get a df per team per date. add **result _column_** for that team
### 3 add neccessary **flag _columns_** and **game rank _column_**. keep **win/loss _columns_** as na if untrue
### 4 group and agregated the **falg _columns_**
### 5 format. create **streak _column_**
### 6 crreate **rank _column_**

In [3]:
#step 1
import pandas as pd
import numpy as np
from pandasgui import show
from datetime import datetime as dt

Excelfile = pd.ExcelFile('2020 week 3/PD - NBA Results (1).xlsx')
Teams = pd.read_excel(Excelfile, sheet_name='Team List')

sheet_to_df_map = {}
for sheet_name in Excelfile.sheet_names:
    if 'Results' in sheet_name:
        sheet_to_df_map[sheet_name] = Excelfile.parse(sheet_name)

results = pd.concat(sheet_to_df_map,ignore_index=True)
results = results.drop(['Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Notes', 'Start (ET)'], axis=1)
results = results.rename(columns={'PTS.1': 'Home Points', 'Visitor/Neutral': 'Visiting Team', 'PTS': 'Visiting Points', 'Home/Neutral': 'Home Team'})
results = results[results['Home Points'].notna()]

results['Winner'] = np.where(results['Visiting Points'] > results['Home Points'], results['Visiting Team'], results['Home Team'])
results['Loser'] = np.where(results['Visiting Points'] > results['Home Points'], results['Home Team'], results['Visiting Team'])

results = results.merge(Teams, left_on='Winner', right_on='Team').drop(columns=['Team','Divison']).rename(columns={'Conference':'Winner Conference'})
results = results.merge(Teams, left_on='Loser', right_on='Team').drop(columns=['Team','Divison']).rename(columns={'Conference':'Loser Conference'})
results['In Conf?'] = np.where(results['Winner Conference'] == results['Loser Conference'],1,0)
results

,Date,Visiting Team,Visiting Points,Home Team,Home Points,Winner,Loser,Winner Conference,Loser Conference,In Conf?
0,Tue Oct 22 2019,New Orleans Pelicans,122.0,Toronto Raptors,130.0,Toronto Raptors,New Orleans Pelicans,Eastern,Western,0
1,Fri Nov 8 2019,Toronto Raptors,122.0,New Orleans Pelicans,104.0,Toronto Raptors,New Orleans Pelicans,Eastern,Western,0
2,Sun Nov 24 2019,New Orleans Pelicans,109.0,Los Angeles Clippers,134.0,Los Angeles Clippers,New Orleans Pelicans,Western,Western,1
3,Mon Dec 9 2019,Detroit Pistons,105.0,New Orleans Pelicans,103.0,Detroit Pistons,New Orleans Pelicans,Eastern,Western,0
4,Sun Dec 15 2019,Orlando Magic,130.0,New Orleans Pelicans,119.0,Orlando Magic,New Orleans Pelicans,Eastern,Western,0
...,...,...,...,...,...,...,...,...,...,...
528,Wed Nov 13 2019,Los Angeles Clippers,93.0,Houston Rockets,102.0,Houston Rockets,Los Angeles Clippers,Western,Western,1
529,Thu Dec 19 2019,Houston Rockets,122.0,Los Angeles Clippers,117.0,Houston Rockets,Los Angeles Clippers,Western,Western,1
530,Sun Dec 22 2019,Los Angeles Clippers,112.0,Oklahoma City Thunder,118.0,Oklahoma City Thunder,Los Angeles Clippers,Western,Western,1
531,Sat Jan 4 2020,Memphis Grizzlies,140.0,Los Angeles Clippers,114.0,Memphis Grizzlies,Los Angeles Clippers,Western,Western,1


In [4]:
#step 2
id_vars = ['Date', 'Home Team', 'Visiting Team',
           'Loser Conference', 'Winner Conference', 'In Conf?']
results = pd.melt(results, id_vars=id_vars,  value_vars=['Winner', 'Loser'],
          var_name='team_type', value_name='Team')
results['conference'] = np.where(results['team_type']=='Winner', results['Winner Conference'], results['Loser Conference'])
results.drop(columns=['Loser Conference', 'Winner Conference'], inplace=True)
results

,Date,Home Team,Visiting Team,In Conf?,team_type,Team,conference
0,Tue Oct 22 2019,Toronto Raptors,New Orleans Pelicans,0,Winner,Toronto Raptors,Eastern
1,Fri Nov 8 2019,New Orleans Pelicans,Toronto Raptors,0,Winner,Toronto Raptors,Eastern
2,Sun Nov 24 2019,Los Angeles Clippers,New Orleans Pelicans,1,Winner,Los Angeles Clippers,Western
3,Mon Dec 9 2019,New Orleans Pelicans,Detroit Pistons,0,Winner,Detroit Pistons,Eastern
4,Sun Dec 15 2019,New Orleans Pelicans,Orlando Magic,0,Winner,Orlando Magic,Eastern
...,...,...,...,...,...,...,...
1061,Wed Nov 13 2019,Houston Rockets,Los Angeles Clippers,1,Loser,Los Angeles Clippers,Western
1062,Thu Dec 19 2019,Los Angeles Clippers,Houston Rockets,1,Loser,Los Angeles Clippers,Western
1063,Sun Dec 22 2019,Oklahoma City Thunder,Los Angeles Clippers,1,Loser,Los Angeles Clippers,Western
1064,Sat Jan 4 2020,Los Angeles Clippers,Memphis Grizzlies,1,Loser,Los Angeles Clippers,Western


In [5]:
#step 3
# flags for summarization
results['home_flag'] = np.where(results['Team']==results['Home Team'], 1, 0)
results['away_flag'] = np.where(results['Team']==results['Visiting Team'], 1, 0)

results['win_flag'] = np.where(results['team_type']=='Winner', 1, np.nan)
results['loss_flag'] = np.where(results['team_type']=='Loser', 1, np.nan)

results['Date'] = pd.to_datetime(results['Date'], infer_datetime_format=True)
results.sort_values(['Team', 'Date'], ascending=False, inplace=True)
results['win_streak'] = results.groupby('Team')['win_flag'].cumsum(skipna=False) # this will count up from one until it hits a NaN
results['loss_streak'] = results.groupby('Team')['loss_flag'].cumsum(skipna=False)


results['win_flag'] = results['win_flag'].fillna(0)
results['loss_flag'] = results['loss_flag'].fillna(0)

results['conference_win_flag'] = np.where((results['team_type']=='Winner') & (results['In Conf?']==1), 1, 0)
results['conference_loss_flag'] = np.where((results['team_type']=='Loser') & (results['In Conf?']==1), 1, 0)

results['home_win_flag'] = np.where((results['team_type']=='Winner') & (results['home_flag']==1), 1, 0)
results['home_loss_flag'] = np.where((results['team_type']=='Loser') & (results['home_flag']==1), 1, 0)

results['away_win_flag'] = np.where((results['team_type']=='Winner') & (results['away_flag']==1), 1, 0)
results['away_loss_flag'] = np.where((results['team_type']=='Loser') & (results['away_flag']==1), 1, 0)

results['date_rank'] = results.groupby('Team')['Date'].rank(method='first', ascending=False)
results['L10_win_flag'] = np.where((results['team_type']=='Winner') & (results['date_rank']<=10), 1, 0)
results['L10_loss_flag'] = np.where((results['team_type']=='Loser') & (results['date_rank']<=10), 1, 0)
results

,Date,Home Team,Visiting Team,In Conf?,team_type,Team,conference,home_flag,away_flag,win_flag,...,loss_streak,conference_win_flag,conference_loss_flag,home_win_flag,home_loss_flag,away_win_flag,away_loss_flag,date_rank,L10_win_flag,L10_loss_flag
515,2020-01-04,Washington Wizards,Denver Nuggets,0,Winner,Washington Wizards,Eastern,1,0,1.0,...,NaN,0,0,1,0,0,0,1.0,1,0
835,2020-01-03,Washington Wizards,Portland Trail Blazers,0,Loser,Washington Wizards,Eastern,1,0,0.0,...,NaN,0,0,0,1,0,0,2.0,0,1
826,2020-01-01,Washington Wizards,Orlando Magic,1,Loser,Washington Wizards,Eastern,1,0,0.0,...,NaN,0,1,0,1,0,0,3.0,0,1
500,2019-12-30,Washington Wizards,Miami Heat,1,Winner,Washington Wizards,Eastern,1,0,1.0,...,NaN,1,0,1,0,0,0,4.0,1,0
840,2019-12-28,Washington Wizards,New York Knicks,1,Loser,Washington Wizards,Eastern,1,0,0.0,...,NaN,0,1,0,1,0,0,5.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,2019-10-31,Atlanta Hawks,Miami Heat,1,Loser,Atlanta Hawks,Eastern,1,0,0.0,...,NaN,0,1,0,1,0,0,32.0,0,0
701,2019-10-29,Miami Heat,Atlanta Hawks,1,Loser,Atlanta Hawks,Eastern,0,1,0.0,...,NaN,0,1,0,0,0,1,33.0,0,0
704,2019-10-28,Atlanta Hawks,Philadelphia 76ers,1,Loser,Atlanta Hawks,Eastern,1,0,0.0,...,NaN,0,1,0,1,0,0,34.0,0,0
56,2019-10-26,Atlanta Hawks,Orlando Magic,1,Winner,Atlanta Hawks,Eastern,1,0,1.0,...,NaN,1,0,1,0,0,0,35.0,0,0


In [7]:
#step 5
results_summary = results.groupby(['conference','Team'], as_index=False).agg(
               { 'win_flag' : 'sum',
                 'loss_flag' : 'sum',
                 'conference_win_flag' : 'sum',
                 'conference_loss_flag' : 'sum',
                 'home_win_flag' : 'sum',
                 'home_loss_flag' : 'sum',
                 'away_win_flag' : 'sum',
                 'away_loss_flag' : 'sum',
                 'L10_win_flag' : 'sum',
                 'L10_loss_flag' : 'sum',
                 'win_streak' : 'max',
                 'loss_streak' :  'max'
               }
             ).rename(columns=
               { 'win_flag' : 'W',
                 'loss_flag' : 'L',
                 'conference_win_flag' : 'Conf_W',
                 'conference_loss_flag' : 'Conf_L' ,
                 'home_win_flag' : 'Home_W' ,
                 'home_loss_flag' : 'Home_L' ,
                 'away_win_flag' : 'Away_W' ,
                 'away_loss_flag' : 'Away_L',
                 'L10_win_flag' : 'L10_W' ,
                 'L10_loss_flag' : 'L10_L' ,
                 'win_streak' : 'Strk_W' ,
                 'loss_streak' : 'Strk_L'
               }
             )
results_summary

,conference,Team,W,L,Conf_W,Conf_L,Home_W,Home_L,Away_W,Away_L,L10_W,L10_L,Strk_W,Strk_L
0,Eastern,Atlanta Hawks,8.0,28.0,5,19,4,12,4,16,2,8,1.0,NaN
1,Eastern,Boston Celtics,25.0,8.0,18,5,14,2,11,6,8,2,3.0,NaN
2,Eastern,Brooklyn Nets,16.0,18.0,10,9,9,7,7,11,3,7,NaN,5.0
3,Eastern,Charlotte Hornets,15.0,23.0,10,14,7,11,8,12,3,7,2.0,NaN
4,Eastern,Chicago Bulls,13.0,23.0,9,15,7,13,6,10,4,6,NaN,3.0
5,Eastern,Cleveland Cavaliers,10.0,25.0,6,21,6,12,4,13,4,6,NaN,3.0
6,Eastern,Detroit Pistons,13.0,23.0,9,18,8,10,5,13,2,8,1.0,NaN
7,Eastern,Indiana Pacers,22.0,14.0,15,10,15,4,7,10,5,5,NaN,2.0
8,Eastern,Miami Heat,25.0,10.0,18,4,16,1,9,9,7,3,NaN,1.0
9,Eastern,Milwaukee Bucks,32.0,5.0,20,3,18,2,14,3,8,2,5.0,NaN


In [8]:
#step 6
# remove the extra column index level and convert numbers to integer
# results_summary.columns = [c[0] if c[1]=='' else c[1] for c in results_summary.columns]
results_summary['Strk_W'] = results_summary['Strk_W'].fillna(0)
results_summary['Strk_L'] = results_summary['Strk_L'].fillna(0)
# print(results_summary)


results_summary['Pct'] = (results_summary['W'] / (results_summary['W'] + results_summary['L'])).round(3)
results_summary['Conf'] = results_summary['Conf_W'].astype(str) + '-' + results_summary['Conf_L'].astype(str)
results_summary['Home'] = results_summary['Home_W'].astype(str) + '-' + results_summary['Home_L'].astype(str)
results_summary['Away'] = results_summary['Away_W'].astype(str) + '-' + results_summary['Away_L'].astype(str)
results_summary['L10'] = results_summary['L10_W'].astype(str) + '-' + results_summary['L10_L'].astype(str)
results_summary['Strk'] = np.where(results_summary['Strk_W']==0, 'L' + results_summary['Strk_L'].astype(str),
                                                    'W' + results_summary['Strk_W'].astype(str))


# conference rank
results_summary['Rank'] = results_summary.groupby('conference')['Pct'].rank(method='max', ascending=False)
results_summary.sort_values(['conference', 'Rank'], ascending=True, inplace=True)

results_summary

,conference,Team,W,L,Conf_W,Conf_L,Home_W,Home_L,Away_W,Away_L,...,L10_L,Strk_W,Strk_L,Pct,Conf,Home,Away,L10,Strk,Rank
9,Eastern,Milwaukee Bucks,32.0,5.0,20,3,18,2,14,3,...,2,5.0,0.0,0.865,20-3,18-2,14-3,8-2,W5.0,1.0
1,Eastern,Boston Celtics,25.0,8.0,18,5,14,2,11,6,...,2,3.0,0.0,0.758,18-5,14-2,11-6,8-2,W3.0,2.0
8,Eastern,Miami Heat,25.0,10.0,18,4,16,1,9,9,...,3,0.0,1.0,0.714,18-4,16-1,9-9,7-3,L1.0,3.0
13,Eastern,Toronto Raptors,24.0,12.0,17,7,14,5,10,7,...,4,1.0,0.0,0.667,17-7,14-5,10-7,6-4,W1.0,4.0
12,Eastern,Philadelphia 76ers,23.0,14.0,16,8,16,2,7,12,...,7,0.0,4.0,0.622,16-8,16-2,7-12,3-7,L4.0,5.0
7,Eastern,Indiana Pacers,22.0,14.0,15,10,15,4,7,10,...,5,0.0,2.0,0.611,15-10,15-4,7-10,5-5,L2.0,6.0
2,Eastern,Brooklyn Nets,16.0,18.0,10,9,9,7,7,11,...,7,0.0,5.0,0.471,10-9,9-7,7-11,3-7,L5.0,7.0
11,Eastern,Orlando Magic,16.0,20.0,11,11,11,8,5,12,...,6,0.0,1.0,0.444,11-11,11-8,5-12,4-6,L1.0,8.0
3,Eastern,Charlotte Hornets,15.0,23.0,10,14,7,11,8,12,...,7,2.0,0.0,0.395,10-14,7-11,8-12,3-7,W2.0,9.0
4,Eastern,Chicago Bulls,13.0,23.0,9,15,7,13,6,10,...,6,0.0,3.0,0.361,9-15,7-13,6-10,4-6,L3.0,11.0
